# Summary of the Code for generating Strain images

This script simulates the stretching and recovery of an image by generating a sequence of images that illustrate both expansion (stretching) and contraction (shrinking) along the horizontal (`x`) and vertical (`y`) axes. The image is padded with a white frame to prevent clipping during transformations. The sequence of images is saved in a specified output directory.

## Functions Overview

### `add_white_frame(image, max_strain)`
Adds a white frame around the image to accommodate maximum strain in both width and height. This ensures that the image can expand without any clipping.

**Parameters:**
- `image`: Original image (PIL.Image object).
- `max_strain`: Maximum strain (float).

**Returns:**
- PIL.Image object with a white frame.

### `stretch_image(image, strain, step, total_steps, axis, shrink=False)`
Stretches or shrinks the image based on the current step and the specified strain value. The transformation occurs along the given axis (`x` or `y`), and the image is resized accordingly. A white background is added to ensure the transformed image fits within the same dimensions.

**Parameters:**
- `image`: Original image (PIL.Image object).
- `strain`: Maximum strain (float).
- `step`: Current step (int).
- `total_steps`: Total number of steps (int).
- `axis`: Axis of strain ('x' for width, 'y' for height).
- `shrink`: Whether to shrink the image (if `True`, it shrinks instead of stretching).

**Returns:**
- Stretched or shrunk PIL.Image object.

### `simulate_image_stretching(image_path, base_output_dir, max_strain, desired_images)`
The main function that performs the image stretching and recovery simulation. It creates a series of images with the following steps:
- Add a consistent white frame to the original image.
- Stretch the image along the `x` axis (expansion), then recover it back to the original size.
- Stretch and recover the image along the `y` axis.
- Generate 3 identical copies of the final image.
- Save all the images in a directory named with the current date and the desired number of images.

**Parameters:**
- `image_path`: Path to the input image (str).
- `base_output_dir`: Base directory to save the output images (str).
- `max_strain`: Maximum strain (float).
- `desired_images`: Desired total number of images to be generated (int).

## Code Workflow

1. **Input Image**: The script starts by loading the input image and calculating the number of steps required for each phase of stretching and recovery.
   
2. **Image Transformation**: The image undergoes stretching and shrinking both horizontally (x-axis) and vertically (y-axis), with each step creating a new image.
   
3. **White Frame**: To avoid clipping, a white frame is added around the image during all transformations.

4. **Output**: A unique output folder is created using the current date, and the images are saved sequentially in that folder.

5. **Final Images**: After all transformations, the last image is duplicated three times and saved to the output directory.

In [4]:
from PIL import Image, ImageOps  # Importing necessary libraries for image processing
import os
from datetime import datetime  # For adding timestamp to output directories

# Function to add a white frame to center the image with padding on all edges
def add_white_frame(image, max_strain):
    """
    Adds a white frame around the image to accommodate the maximum strain in both directions.

    Parameters:
    - image: Original image (PIL.Image object).
    - max_strain: Maximum strain (float), determines how much the image can expand.

    Returns:
    - PIL.Image object with white frame.
    """
    width, height = image.size  # Get the current size of the image
    
    # Calculate the new size of the image after applying the maximum strain
    max_width = int(width * (1 + max_strain))  # New width after strain
    max_height = int(height * (1 + max_strain))  # New height after strain
    
    # Calculate how much padding is needed for each edge
    padding_width = (max_width - width) // 2
    padding_height = (max_height - height) // 2
    
    # Create a new image with white padding
    new_image = ImageOps.expand(image, border=(padding_width, padding_height, padding_width, padding_height), fill="white")
    return new_image

# Function to stretch or shrink an image
def stretch_image(image, strain, step, total_steps, axis, shrink=False):
    """
    Stretches or shrinks the image based on the current step and strain along the specified axis.

    Parameters:
    - image: Original image (PIL.Image object).
    - strain: Maximum strain (float), determines how much to stretch or shrink the image.
    - step: Current step (int), used to calculate the stretch factor.
    - total_steps: Total number of steps (int), used to normalize the stretch factor.
    - axis: Axis of strain ('x' for width, 'y' for height).
    - shrink: Whether to shrink the image (if True, it shrinks instead of stretching).

    Returns:
    - Stretched or shrunk PIL.Image object.
    """
    # Calculate the stretch factor based on whether we're stretching or shrinking
    if shrink:
        stretch_factor = 1 - strain * (step / total_steps)  # Shrinking logic
    else:
        stretch_factor = 1 + strain * (step / total_steps)  # Stretching logic

    width, height = image.size  # Get the current dimensions of the image

    # Apply the stretch to the specified axis
    if axis == 'x':
        new_width = int(width * stretch_factor)  # Stretch/shrink width
        transformed = image.resize((new_width, height), Image.Resampling.BICUBIC)  # Rescale the image
    elif axis == 'y':
        new_height = int(height * stretch_factor)  # Stretch/shrink height
        transformed = image.resize((width, new_height), Image.Resampling.BICUBIC)  # Rescale the image
    else:
        raise ValueError("Axis must be either 'x' or 'y'")  # Handle invalid axis input

    # Create a new image with a white background to paste the transformed image into
    framed_image = Image.new("RGB", (width, height), "white")
    
    # Calculate the offset to center the transformed image within the white frame
    x_offset = (width - transformed.size[0]) // 2
    y_offset = (height - transformed.size[1]) // 2
    
    # Paste the transformed image into the white background
    framed_image.paste(transformed, (x_offset, y_offset))

    return framed_image

# Main program that simulates the stretching and recovery of the image
def simulate_image_stretching(image_path, base_output_dir, max_strain, desired_images):
    """
    Simulates the stretching and recovery of an image based on the desired number of images.

    Parameters:
    - image_path: Path to the input image (str).
    - base_output_dir: Base directory to save the output images (str).
    - max_strain: Maximum strain (float), the max amount the image can be stretched or shrunk.
    - desired_images: Desired total number of images to be generated (int).
    """
    # Load the original image
    original_image = Image.open(image_path)

    # Calculate the total number of steps (both expansion and recovery phases)
    steps_per_phase = (desired_images // 4)  # Divide images equally between expansion and recovery phases

    # Get the current date in YYYY-MM-DD format
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Create a unique output directory with the number of images and current date
    output_dir = os.path.join(base_output_dir, f"output_{desired_images}_{current_date}")
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Add a consistent white frame to the original image to prevent clipping during transformations
    framed_image = add_white_frame(original_image, max_strain)

    # Track the image step for naming files
    step_counter = 1

    # Stretch and recover along the x-axis (expansion and recovery)
    axis = 'x'
    for step in range(steps_per_phase + 1):  # Expansion phase for x-axis
        stretched_image = stretch_image(framed_image, max_strain, step, steps_per_phase, axis)
        output_path = os.path.join(output_dir, f"strain_{step_counter:03d}.jpg")
        stretched_image.save(output_path, format="JPEG")  # Save the stretched image
        step_counter += 1

    # Recover along x-axis (shrink back to original size)
    for step in range(1, steps_per_phase + 1):  # Recovery phase for x-axis
        recovered_image = stretch_image(framed_image, max_strain, steps_per_phase - step, steps_per_phase, axis)
        output_path = os.path.join(output_dir, f"strain_{step_counter:03d}.jpg")
        recovered_image.save(output_path, format="JPEG")  # Save the recovered image
        step_counter += 1

    # Reload the framed image to ensure consistency for y-axis transformations
    framed_image = add_white_frame(original_image, max_strain)

    # Shrink and recover along the y-axis (shrinking and recovery)
    axis = 'y'
    for step in range(steps_per_phase + 1):  # Shrinking phase for y-axis
        transformed_image = stretch_image(framed_image, max_strain, step, steps_per_phase, axis, shrink=True)
        output_path = os.path.join(output_dir, f"strain_{step_counter:03d}.jpg")
        transformed_image.save(output_path, format="JPEG")  # Save the shrunk image
        step_counter += 1

    # Recover along y-axis (expand back to original size)
    for step in range(1, steps_per_phase + 1):  # Recovery phase for y-axis
        recovered_image = stretch_image(framed_image, max_strain, steps_per_phase - step, steps_per_phase, axis, shrink=True)
        output_path = os.path.join(output_dir, f"strain_{step_counter:03d}.jpg")
        recovered_image.save(output_path, format="JPEG")  # Save the recovered image
        step_counter += 1

    # Generate 3 copies of the last image to fill out the desired image count
    last_image_path = os.path.join(output_dir, f"strain_{step_counter - 1:03d}.jpg")
    last_image = Image.open(last_image_path)

    for i in range(3):  # Duplicate the last image 3 times
        output_path = os.path.join(output_dir, f"strain_{step_counter:03d}.jpg")
        last_image.save(output_path, format="JPEG")
        step_counter += 1

    # Print a completion message with the output directory
    print(f"Simulation completed. Images saved in '{output_dir}'.")






## User-based parameter definition

In [ ]:
# User Input
image_path = "input.jpg"  # Path to your input image (replace with actual path)
base_output_dir = "output_images"  # Base directory to save output images
max_relative_strain = 0.20  # Maximum strain (e.g., ±20% max stretch/shrink)
desired_images = 30  # Specify the desired number of images to generate

simulate_image_stretching(image_path, base_output_dir, max_relative_strain, desired_images)

Simulation completed. Images saved in 'output_images\output_30_2025-04-14'.
